In [17]:
import pandas as pd
from itertools import permutations

In [18]:
df = pd.read_csv('prozhito_predictions.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'preds', 'lens'])
df['bio'] = df['bio'].apply(lambda x: eval(x))
df['bio_sum'] = df['bio'].apply(lambda bio: len([i for i in bio if i.startswith('B')]))

## Defining Rules №1: Possible Connections

In [21]:
connections = {
    ('PER', 'PER'),
    ('PER', 'LOC'),
    ('PER', 'FAC'),
    ('PER', 'ORG'),
    ('PER', 'CHAR'),
    ('ORG', 'LOC'),
    ('FAC', 'LOC')
}

In [22]:
df['bio_unique'] = df['bio'].apply(lambda x: [i[2:] for i in x if i != 'O'])

In [23]:
df.head(2)

,text,bio,bio_sum,bio_unique
0,> « Отрекись ! » — но восемь лет монахи > Слыш...,"[O, O, O, O, O, O, O, O, O, O, O, B-CHAR, O, O...",4,"[CHAR, PER, PER, PER]"
1,"> Ты был прав , Ноламец . > Ты не умел кривить...","[O, O, O, O, O, B-PER, B-PER, B-PER, O, O, O, ...",3,"[PER, PER, PER]"


In [24]:
df['conn_num'] = df['bio_unique'].apply(lambda bio: len(connections & set(permutations(tuple(bio), 2))))

In [25]:
df_filtered = df[df['conn_num'] > 0]
df_filtered

,text,bio,bio_sum,bio_unique,conn_num
0,> « Отрекись ! » — но восемь лет монахи > Слыш...,"[O, O, O, O, O, O, O, O, O, O, O, B-CHAR, O, O...",4,"[CHAR, PER, PER, PER]",2
1,"> Ты был прав , Ноламец . > Ты не умел кривить...","[O, O, O, O, O, B-PER, B-PER, B-PER, O, O, O, ...",3,"[PER, PER, PER]",1
2,> Потомки бы тебе простили > ( Они ж простили ...,"[O, O, O, O, O, O, O, O, O, O, O, O, B-PER, B-...",2,"[PER, PER]",1
3,> Сгорев — ты ими стал . > И если б ты ожил > ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[CHAR, PER, PER]",2
5,Главной достопримечательностью этого села явля...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER,...",4,"[PER, PER, PER, PER, PER, FAC, FAC]",2
...,...,...,...,...,...
56756,"К счастью , 5 марта 1953 года Иосиф Виссарионо...","[O, O, O, O, O, O, O, B-PER, I-PER, B-FAC, O]",2,"[PER, PER, FAC]",2
56758,"Так что , Амосов , не жалуйся на судьбу , было...","[O, O, O, B-PER, B-PER, O, O, O, O, O, O, O, O]",2,"[PER, PER]",1
56761,Есть немалые надежды : Горбачев 7 декабря встр...,"[O, O, O, O, B-PER, O, O, O, O, B-PER, B-PER]",3,"[PER, PER, PER]",1
56762,"Министр Чазов все понимает , но денег у него н...","[B-CHAR, B-PER, B-PER, O, O, O, O, O, O, O, O]",3,"[CHAR, PER, PER]",2


## Defining Rules №2: Key Words Extraction

In [31]:
key_words = '''брат сестр отец отц мать матер мам супруг жен муж родил рожд 
родит жил умер член религ католи христ будди атеист 
русск америк поляч поляк англич 
художн писате поэт организован создал создат химик физик яредщ
азербайдж армян армен белорус беларус груз казах киргиз латв литов молдав 
таджик туркмен узбекск украин эстон
'''

key_words = [' ' + w for w in key_words.split()]
len(key_words)

51

In [32]:
df_filtered_2 = df_filtered[df_filtered['text'].apply(lambda text: len([w for w in key_words if w in text])) > 0].reset_index()
df_filtered_2

,index,text,bio,bio_sum,bio_unique,conn_num
0,0,> « Отрекись ! » — но восемь лет монахи > Слыш...,"[O, O, O, O, O, O, O, O, O, O, O, B-CHAR, O, O...",4,"[CHAR, PER, PER, PER]",2
1,54,В этих созвездиях звездой первой величины был ...,"[O, O, O, O, O, O, O, O, B-PER, I-PER, B-FAC, ...",2,"[PER, PER, FAC]",2
2,76,"Если Достоевский душу русскую , её широту , си...","[O, B-PER, O, O, O, O, O, O, O, O, O, O, O, O,...",3,"[PER, PER, PER]",1
3,88,"Пьер Лапезан де Буагельбер < com id = "" 155248...","[B-PER, I-PER, B-FAC, B-FAC, I-PER, B-FAC, B-L...",6,"[PER, PER, FAC, FAC, PER, FAC, LOC, FAC]",4
4,119,Но два случая доказывают обратное — душа в Аду...,"[O, O, O, O, O, O, O, O, B-PER, B-PER, B-PER, O]",3,"[PER, PER, PER]",1
...,...,...,...,...,...,...
5296,56663,Этот раз подвернулась пленка об алкоголизме не...,"[O, O, O, O, O, O, O, O, O, O, B-PER, B-PER, O]",2,"[PER, PER]",1
5297,56672,Во время конференции Аня подала записку : « В ...,"[O, O, O, B-PER, O, O, O, O, O, O, O, O, O, B-...",3,"[PER, PER, PER, PER, FAC]",2
5298,56702,"Мой друг Григорий Иосифович Кипнис , писатель ...","[O, O, B-PER, I-PER, I-PER, B-FAC, O, O, O, O,...",2,"[PER, PER, PER, FAC]",2
5299,56711,Анна Васильевна ( жена ) и Таня ( дочь ) делал...,"[B-PER, I-PER, O, B-CHAR, O, O, B-PER, O, B-CH...",4,"[PER, PER, CHAR, PER, CHAR]",2
